### Query OpenAlex for all works authored by a person
This notebook queries the [OpenAlex API](https://docs.openalex.org/api) via its `/works` endpoint for all works authored by a person. It takes an ORCID URL as input which is used to filter for all works where '`authorships.author.orcid`' matches the given ORCID URL.
From the resulting list of works we output their respective DOIs.

In [1]:
# Prerequisite: needed dependency to make HTTP calls
import requests

The input for the query is an ORCID URL.

In [2]:
# input parameter
example_orcid="https://orcid.org/0000-0003-2499-7741"

We use it to query the OpenAlex API for works that specified the ORCID URL within their metadata in the field '`authorships.author.orcid`'.
 Since the OpenAlex API uses [pagination](https://docs.openalex.org/api/get-lists-of-entities#pagination), we need to loop through all pages to get the complete result set.

In [3]:
# OpenAlex endpoint to query for works
OPENALEX_API_WORKS = "https://api.openalex.org/works"

# query all works that are connected to orcid
def download_data(orcid):
    page = 1
    max_page = 1
    while page <= max_page:
        params = {'filter': 'authorships.author.orcid:'+orcid, 'page': page}

        response = requests.get(url=OPENALEX_API_WORKS,
                                params=params,
                                headers= {'Content-Type': 'application/json'})
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
          max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil


# ---- example execution
list_of_pages=download_data(example_orcid)

From the resulting list of works we extract and print out title and DOI.

In [4]:
# extract all DOIs from the result
def extract_dois(data):
    for work in data['results']:
      try:
        doi=work['ids']['doi'].replace("https://doi.org/", "")
        title=work['display_name']
        yield doi, title
      except KeyError:
        pass


# ---- example execution
for page in list_of_pages:
    for doi,title in extract_dois(page):
        print(f"{doi}, {title}")

10.3897/rio.7.e66264, OPTIMETA – Strengthening the Open Access publishing system through open citations and spatiotemporal metadata
10.11588/ip.2020.2.73938, Bericht vom 4. VIVO-Workshop 2019
10.15488/9424, ConfIDent – An Open Platform for FAIR Conference Metadata
10.31263/voebm.v72i2.2808, Open Science und die Bibliothek – Aktionsfelder und Berufsbild
10.21105/joss.01182, VIVO: a system for research discovery
10.11588/ip.2019.1.61729, Bericht vom 3. VIVO-Workshop 2018
10.11588/ip.2019.1.49609, Problematische Aspekte bibliometrie-basierter Forschungsevaluierung
10.1016/j.procs.2019.01.074, The Research Core Dataset (KDSF) in the Linked Data context
10.11588/ip.2018.1.49357, Vitro - ein universell einsetzbarer Editor für Ontologien und Instanzen
10.3897/rio.4.e31656, Reference implementation for open scientometric indicators (ROSI)
10.5281/zenodo.1464108, VIVO - eine Einführung
10.11588/ip.2018.1.46819, Anforderungen an Forschungsinformationssysteme in Deutschland durch Forschende und F